In [2]:
!pip install ipywidgets matplotlib --quiet

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Button
from IPython.display import display, clear_output

# --- Αρχικοποίηση παραμέτρων ---
M = 50  # Διαστάσεις πλέγματος (MxM)

# Δυνατότητες του εδάφους (χωρητικότητα)
CN, CP, CK = 1.0, 1.0, 1.0  # max concentration

# Ρυθμοί διάχυσης
DN, DP, DK = 0.1, 0.05, 0.07

# Θερμοκρασία και υγρασία σταθερές
Ts = 25  # Celsius
Hs = 0.6  # 0 to 1

# --- Δημιουργία πλέγματος ---
def initialize_grid(granules=50):
    grid = np.zeros((M, M, 5))  # [cn, cp, ck, hs, ts]
    grid[:, :, 3] = Hs
    grid[:, :, 4] = Ts
    
    for _ in range(granules):
        x, y = np.random.randint(0, M), np.random.randint(0, M)
        grid[x, y, 0] += 0.5  # N
        grid[x, y, 1] += 0.5  # P
        grid[x, y, 2] += 0.5  # K
    return grid

# --- Διάχυση με χρήση μέσης τιμής γειτονικών κυψελίδων ---
def diffuse(grid, steps=10):
    for _ in range(steps):
        new_grid = grid.copy()
        for i in range(1, M-1):
            for j in range(1, M-1):
                for k, D in enumerate([DN, DP, DK]):
                    neighborhood = grid[i-1:i+2, j-1:j+2, k]
                    diffusion = D * (np.mean(neighborhood) - grid[i, j, k])
                    new_grid[i, j, k] += diffusion
                    new_grid[i, j, k] = min(new_grid[i, j, k], [CN, CP, CK][k])
        grid = new_grid
    return grid

# --- Οπτικοποίηση διάχυσης ---
def plot_diffusion(grid):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    labels = ['N (Azote)', 'P (Phosphorus)', 'K (Potassium)']
    for i in range(3):
        axs[i].imshow(grid[:, :, i], cmap='viridis')
        axs[i].set_title(labels[i])
        axs[i].axis('off')
    plt.show()

# --- GUI με sliders ---
def run_simulation(granules, steps, Dn, Dp, Dk):
    global DN, DP, DK
    DN, DP, DK = Dn, Dp, Dk
    grid = initialize_grid(granules)
    final = diffuse(grid, steps)
    plot_diffusion(final)

# GUI controls
interact(
    run_simulation,
    granules=IntSlider(min=10, max=200, step=10, value=50, description='Κόκκοι'),
    steps=IntSlider(min=1, max=50, step=1, value=10, description='Βήματα'),
    Dn=FloatSlider(min=0.01, max=0.3, step=0.01, value=0.1, description='Διάχυση N'),
    Dp=FloatSlider(min=0.01, max=0.3, step=0.01, value=0.05, description='Διάχυση P'),
    Dk=FloatSlider(min=0.01, max=0.3, step=0.01, value=0.07, description='Διάχυση K')
);


interactive(children=(IntSlider(value=50, description='Κόκκοι', max=200, min=10, step=10), IntSlider(value=10,…